<a href="https://colab.research.google.com/github/youssefjedidi/Aircraft_Noise_Predictor/blob/main/aircraft_noise_predictor_ML_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Regression models

## Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [ ]:
dataset = pd.read_csv('aircraft_noise.csv')
X = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

## Encoding categorical data

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [  1 , 0 ,3 ])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Multiple Linear Regression


### Training the Multiple Linear Regression model on the Training set

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

### Predicting the Test set results

In [ ]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

### Testing accuracy

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return (np.mean(np.abs((y_true - y_pred) / y_true)) * 100)

mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(regressor.predict([[1, 0, 0, 1 ,0 ,0 , 0,0 ,1 , 300]]))

from sklearn.metrics import r2_score

# Assuming y_test contains the actual target values and y_pred contains the predicted values
# Calculate R squared
r2 = r2_score(y_test, y_pred)

# Print MSE
print(f" R squared  (r2): {r2:.2f}")

## Polynomial Regression

### Training

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 3)
X_poly = poly_reg.fit_transform(X_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)

### Predecting the Test set Results

In [ ]:
y_poly = lin_reg_2.predict(poly_reg.fit_transform(X_test))
np.set_printoptions(precision=2)
print(np.concatenate((y_poly.reshape(len(y_poly),1), y_test.reshape(len(y_test),1)),1))

### Testing accuracy

In [ ]:

mape = mean_absolute_percentage_error(y_test, y_poly)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(lin_reg_2.predict(poly_reg.fit_transform([[1, 0, 0, 1 ,0 ,0 , 0,0 ,1 , 300]])))

from sklearn.metrics import r2_score

# Assuming y_test contains the actual target values and y_pred contains the predicted values
# Calculate R squared
r2_poly = r2_score(y_test, y_poly)

# Print MSE
print(f" R squared  (r2): {r2_poly:.2f}")

## Support Vector Regression (SVR)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

# Reshape input data to 2D array
X_reshaped = X[:, 3].reshape(-1, 1)  # Reshape only the fourth column
X_test_reshaped = X_test[:, 3].reshape(-1, 1)

# Initialize StandardScaler for the fourth column
sc_X = StandardScaler()

# Fit and transform the fourth column
X_scaled = X.copy()  # Create a copy of X to keep other columns unchanged
X_scaled[:, 3] = sc_X.fit_transform(X_reshaped).flatten()  # Scale and update the fourth column

X_test_scaled = X_test.copy()
X_test_scaled[:, 3] = sc_X.fit_transform(X_test_reshaped).flatten()

# Initialize StandardScaler for the target variable
sc_y = StandardScaler()
y_scaled = sc_y.fit_transform(y.reshape(-1, 1))  # Fit and transform the target variable
y_test_scaled = sc_y.fit_transform(y_test.reshape(-1, 1))

# Now X_scaled and y_scaled are scaled versions of X and y using StandardScaler
print("Scaled X:")
print(X_scaled)
print("\nScaled y:")
print(y_scaled)

### Training the SVR model

In [ ]:
from sklearn.svm import SVR
sv_regressor = SVR(kernel = 'poly')
sv_regressor.fit(X_scaled, y_scaled)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Predicting

In [ ]:
# Perform predictions using the SVR model on the scaled test features
y_svr_scaled = sv_regressor.predict(X_test_scaled)

# Inverse transform the scaled predictions to get them back to the original scale
y_svr = sc_y.inverse_transform(y_svr_scaled.reshape(-1, 1))

np.set_printoptions(precision=2)
print(np.concatenate((y_svr.reshape(len(y_svr),1), y_test.reshape(len(y_test),1)),1))

### Testing accuracy

In [ ]:

mape = mean_absolute_percentage_error(y_test, y_svr)

print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

from sklearn.metrics import r2_score

# Assuming y_test contains the actual target values and y_pred contains the predicted values
# Calculate R squared
r2_poly = r2_score(y_test, y_svr)

# Print MSE
print(f" R squared  (r2): {r2_poly:.2f}")

## Random Forest

### Training the Random model on the Training set

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
regressor.fit(X, y)